# TP1 - Finding Keys using Functional Dependencies 
--------------------------

In this lab, we are going to learn 

- how to use Jupyter
- how to use SQLite
- how to discover keys in relations

## How to use Jupyter

To execute a cell, click on it, and then use SHIFT+ENTER (this will execute the contents of the cell, and move down to the next one!)

To edit a cell, click on it. If the cell uses markdown code, then use ENTER to edit it.

See the Help menu for other useful keyboard commands. You can always use the menu bar instead as well.


In [46]:
print("Hello world!")

Hello world!


Another example:

In [47]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


#### Exercise 1

Print numbers 1 to 9 in reverse order

In [48]:
for i in range(9,0,-1) :
    print(i)

9
8
7
6
5
4
3
2
1


## How to use SQLite

To work with SQL easily in a notebook, we'll load the ipython-sql extension as follows:

In [49]:
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


We are going to create a table:

In [50]:
%%sql DROP TABLE IF EXISTS T;
CREATE TABLE T(course VARCHAR, classroom INT, time INT);
INSERT INTO T VALUES ('CS 364', 132, 900);
INSERT INTO T VALUES ('CS 245', 140, 1000);
INSERT INTO T VALUES ('EE 101', 210, 900);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Let's run our first SQL query:

In [51]:
%sql SELECT * FROM T LIMIT 3;

 * sqlite://
Done.


course,classroom,time
CS 364,132,900
CS 245,140,1000
EE 101,210,900


#### Exercise 2

List the name of the courses with time less than 950

In [52]:
%sql SELECT course FROM T WHERE time<950

 * sqlite://
Done.


course
CS 364
EE 101


## How to discover keys in relations

Now, we are going to work with functional dependencies, keys and closures. Our final goal is going to build a method to find keys in a relation.

### Functional Dependencies

Recall that given a set of attributes  $\{A_1, \dots, A_n\}$ and a set of FDs $\Gamma$

The closure, denoted $\{A_1, \dots, A_n\}^+$, is defined to be the largest set of attributes B s.t. $$A_1,\dots,A_n \rightarrow B \text{ using } \Gamma.$$

We're going to use some functions to compute the closure of a set of attributes and other such operations (_from CS145 Stanford_)

In [53]:
# Source code

def to_set(x):
  """Convert input int, string, list, tuple, set -> set"""
  if type(x) == set:
    return x
  elif type(x) in [list, set]:
    return set(x)
  elif type(x) in [str, int]:
    return set([x])
  else:
    raise Exception("Unrecognized type.")

def fd_to_str(lr_tuple): return ",".join(to_set(lr_tuple[0])) + " -> " + ",".join(to_set(lr_tuple[1]))

def fds_to_str(fds): return "\n\t".join(map(fd_to_str, fds))

def set_to_str(x): return "{" + ",".join(x) + "}"

def fd_applies_to(fd, x): 
  lhs, rhs = map(to_set, fd)
  return lhs.issubset(x)

def print_setup(A, fds):
  print("Attributes = " + set_to_str(A))
  print("FDs = \t" + fds_to_str(fds))

def print_fds(fds):
  print("FDs = \t" + fds_to_str(fds))    


Now, let's look at a concrete example. For example, the code for

attributes = { name, category, color, department, price}

and functional dependencies:

name $\rightarrow$ color

category $\rightarrow$ department

color, category $\rightarrow$ price

is the following:

In [54]:
attributes  = set(["name", "category", "color", "department", "price"]) # These are the attribute set.
fds = [(set(["name"]),"color"),
         (set(["category"]), "department"),
         (set(["color", "category"]), "price")]

print_setup(attributes, fds)

Attributes = {price,department,name,color,category}
FDs = 	name -> color
	category -> department
	category,color -> price


### Closure of a set of Attributes

Let's implement the algorithm for obtaining the closure of a set of attributes:

In [55]:
def compute_closure(x, fds, verbose=False):
    bChanged = True        # We will repeat until there are no changes.
    x_ret    = x.copy()    # Make a copy of the input to hold x^{+}
    while bChanged:
        bChanged = False   # Must change on each iteration
        for fd in fds:     # loop through all the FDs.
            (lhs, rhs) = map(to_set, fd) # recall: lhs -> rhs
            if fd_applies_to(fd, x_ret) and not rhs.issubset(x_ret):
                x_ret = x_ret.union(rhs)
                if verbose:
                    print("Using FD " + fd_to_str(fd))
                    print("\t Updated x to " + set_to_str(x_ret))
                bChanged = True
    return x_ret

As an example, let's compute the closure for the attribute "name":

In [56]:
A = set(["name"])
close= compute_closure(A,fds, True)

Using FD name -> color
	 Updated x to {color,name}


#### Exercise 3

Is the attribute "name" a superkey for this relation? Why?

We know that an attribute is a superkey if its closure contains all attributes. Let's verify it using python.

In [57]:
is_superkey = (close==attributes)

if is_superkey :
    print("The attribute 'name' is a superkey !")
else :
    print("The attribute 'name' isn't a superkey...")


The attribute 'name' isn't a superkey...


Hence, its closure doesn't contain all the attributes. Therefore, it is not a superkey.

### Keys and Superkeys

Next, we'll add some new functions now for finding superkeys and keys.  Recall:
* A _superkey_ for a relation $R(B_1,\dots,B_m)$ is a set of attributes $\{A_1,\dots,A_n\}$ s.t.
$$ \{A_1,\dots,A_n\} \rightarrow B_{j} \text{ for all } j=1,\dots m$$
* A _key_ is a minimal (setwise) _superkey_

The algorithm to determine if a set of attributes $A$ is a superkey for $X$ is actually very simple (check if $A^+=X$):

In [58]:
def is_superkey_for(A, X, fds, verbose=False): 
    return X.issubset(compute_closure(A, fds, verbose=verbose))

#### Exercise 4

Is "name" a superkey of the relation?

In [59]:
A = set(["name", "category", "color", "department", "price"])
X = set(["name"])

is_superkey = is_superkey_for(A, X, fds)

if is_superkey :
    print("The attribute 'name' is a superkey !")
else :
    print("The attribute 'name' isn't a superkey...")

The attribute 'name' is a superkey !


Then, to check if $A$ is a key for $X$, we just confirm that:
* (a) it is a superkey
* (b) there are no smaller superkeys (_Note that we only need to check for superkeys of one size smaller_)

In [60]:
import itertools
def is_key_for(A, X, fds, verbose=False):
    subsets = set(itertools.combinations(A, len(A)-1))
    return is_superkey_for(A, X, fds) and \
        all([not is_superkey_for(set(SA), X, fds) for SA in subsets])

Now, let's look at another example:

attributes = { cru, type, client, remise}

and functional dependencies:

cru $\rightarrow$ type

type, client $\rightarrow$ remise

#### Exercise 5

Is "cru" and "client" a key of the relation? Why?

In [61]:
attributes = set(["cru", "type", "client", "remise"])
fds = [(set(["cru"]),"type"),
         (set(["type", "client"]), "remise")]

if is_key_for(set(["cru", "client"]), attributes, fds):
    print("{'cru', 'client'} is a key !")
else:
    print("{'cru', 'client'} not is a key")

{'cru', 'client'} is a key !


Because it's a superkey and it is minimal. 

### Inferring FDs

The algorithm to find the closure of a set of attributes given a set of functional dependencies is the following:

In [62]:
import itertools
def findsubsets(S,m):
    return set(itertools.combinations(S, m))
def closure(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size) 
        for SA in subsets:      # loop through all the subsets.
            cl = compute_closure(set(SA), fds, verbose)
            if len(cl.difference(SA)) > 0: 
                c.extend([(set(SA), cl)])
    return c

Let's see some example of how to use it:

In [64]:
attributes1 = set(['A', 'B', 'C', 'D'])
fds1 = [(set(['A', 'B']), 'C'),
     (set(['A', 'D']), 'B'),
     (set(['B']), 'D')]
closure(attributes1,fds1)

[({'B'}, {'B', 'D'}),
 ({'A', 'D'}, {'A', 'B', 'C', 'D'}),
 ({'A', 'B'}, {'A', 'B', 'C', 'D'}),
 ({'B', 'C'}, {'B', 'C', 'D'}),
 ({'A', 'B', 'D'}, {'A', 'B', 'C', 'D'}),
 ({'A', 'C', 'D'}, {'A', 'B', 'C', 'D'}),
 ({'A', 'B', 'C'}, {'A', 'B', 'C', 'D'})]

Let's remove the attributes for which we are computing the closure from the closure

In [65]:
def closureDifference(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size) 
        for SA in subsets:      # loop through all the subsets.
            cl = compute_closure(set(SA), fds, verbose)
            if len(cl.difference(SA)) > 0: 
                c.extend([(set(SA), cl.difference(SA))])
    return c

Let's see some examples of how to use it to infer the functional dependencies with the previous example:

In [66]:
print_fds(closureDifference(attributes1, fds1))


FDs = 	B -> D
	D,A -> C,B
	B,A -> C,D
	C,B -> D
	B,D,A -> C
	C,D,A -> B
	C,B,A -> D


In [67]:
attributes2 = set (['CRU', 'TYPE', 'CLIENT', 'REMISE'])
fds2 = [(set(['CRU']), 'TYPE'),
     (set(['TYPE', 'CLIENT']), 'REMISE')]
print_fds(closure(attributes2, fds2))

FDs = 	CRU -> TYPE,CRU
	TYPE,CLIENT -> TYPE,CLIENT,REMISE
	CRU,REMISE -> TYPE,CRU,REMISE
	CLIENT,CRU -> REMISE,TYPE,CLIENT,CRU
	REMISE,CLIENT,CRU -> REMISE,TYPE,CLIENT,CRU
	TYPE,CLIENT,CRU -> REMISE,TYPE,CLIENT,CRU


In [68]:
attributes3 = set( ['N VEH', 'TYPE', 'COULEUR', 'MARQUE', 'PUISSANCE'])
fds3 = [(set(['N VEH']), 'TYPE'), 
      (set(['N VEH']), 'COULEUR'),
      (set(['TYPE']), 'MARQUE'),
      (set(['TYPE']), 'PUISSANCE')]
print_fds(closure(attributes3,fds3))

FDs = 	N VEH -> COULEUR,PUISSANCE,TYPE,N VEH,MARQUE
	TYPE -> PUISSANCE,TYPE,MARQUE
	TYPE,MARQUE -> PUISSANCE,TYPE,MARQUE
	TYPE,COULEUR -> MARQUE,PUISSANCE,TYPE,COULEUR
	TYPE,PUISSANCE -> MARQUE,TYPE,PUISSANCE
	N VEH,MARQUE -> COULEUR,PUISSANCE,TYPE,N VEH,MARQUE
	COULEUR,N VEH -> COULEUR,PUISSANCE,TYPE,N VEH,MARQUE
	PUISSANCE,N VEH -> COULEUR,MARQUE,TYPE,N VEH,PUISSANCE
	TYPE,N VEH -> COULEUR,PUISSANCE,TYPE,N VEH,MARQUE
	COULEUR,N VEH,PUISSANCE -> COULEUR,MARQUE,TYPE,N VEH,PUISSANCE
	PUISSANCE,N VEH,MARQUE -> COULEUR,MARQUE,TYPE,N VEH,PUISSANCE
	TYPE,N VEH,COULEUR -> COULEUR,PUISSANCE,TYPE,N VEH,MARQUE
	TYPE,N VEH,MARQUE -> COULEUR,PUISSANCE,TYPE,N VEH,MARQUE
	MARQUE,TYPE,COULEUR -> MARQUE,PUISSANCE,TYPE,COULEUR
	COULEUR,N VEH,MARQUE -> COULEUR,PUISSANCE,TYPE,N VEH,MARQUE
	TYPE,N VEH,PUISSANCE -> COULEUR,MARQUE,TYPE,N VEH,PUISSANCE
	PUISSANCE,TYPE,COULEUR -> MARQUE,PUISSANCE,TYPE,COULEUR
	PUISSANCE,TYPE,N VEH,COULEUR -> COULEUR,TYPE,PUISSANCE,N VEH,MARQUE
	MARQUE,TYPE,N VEH,PUISSANCE ->

Now, let's write a method to find superkeys of the relations:


In [69]:
def superkeys(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size)
        for SA in subsets:
            cl = compute_closure(set(SA), fds, verbose)
            if cl == X and len(cl.difference(SA)) > 0: ## cl = X
                c.extend([SA])
    return c

In [70]:
superkeys(attributes1, fds1)


[('D', 'A'), ('B', 'A'), ('B', 'D', 'A'), ('C', 'D', 'A'), ('C', 'B', 'A')]

Let's see some examples:

In [71]:
superkeys(attributes2, fds2)


[('CLIENT', 'CRU'), ('CLIENT', 'CRU', 'REMISE'), ('CLIENT', 'TYPE', 'CRU')]

In [72]:
superkeys(attributes3, fds3)

[('N VEH',),
 ('N VEH', 'MARQUE'),
 ('COULEUR', 'N VEH'),
 ('N VEH', 'PUISSANCE'),
 ('TYPE', 'N VEH'),
 ('COULEUR', 'N VEH', 'PUISSANCE'),
 ('N VEH', 'PUISSANCE', 'MARQUE'),
 ('TYPE', 'COULEUR', 'N VEH'),
 ('TYPE', 'N VEH', 'MARQUE'),
 ('COULEUR', 'N VEH', 'MARQUE'),
 ('TYPE', 'N VEH', 'PUISSANCE'),
 ('TYPE', 'COULEUR', 'N VEH', 'PUISSANCE'),
 ('TYPE', 'N VEH', 'PUISSANCE', 'MARQUE'),
 ('COULEUR', 'N VEH', 'PUISSANCE', 'MARQUE'),
 ('TYPE', 'COULEUR', 'N VEH', 'MARQUE')]

#### Exercise 6

Implement a method to find the keys of a relation

In [73]:
def keys(attributes, fds, verbose=False):
    superkeys_list = superkeys(attributes, fds, verbose)
    keys = []
    for superkey in superkeys_list:
        is_key = True
        for subset in superkeys_list:
            if subset != superkey and set(subset).issubset(set(superkey)):
                is_key = False
                break
        if is_key:
            keys.append(superkey)
    return keys


Test it 

In [74]:
keys(attributes, fds)

[('client', 'cru')]

In [75]:
keys(attributes2, fds2)

[('CLIENT', 'CRU')]

In [76]:
keys(attributes3, fds3)

[('N VEH',)]